In [6]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 28.6 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd

In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13
current_dir = os.getcwd()
csv_dir = os.path.join(current_dir, 'data', 'raw_csv')
train_csv_path = os.path.join(csv_dir, 'merged_train.csv')

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC_online13


In [5]:
df = pd.read_csv(train_csv_path)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4882 entries, 0 to 4881
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           4882 non-null   int64 
 1   class         4882 non-null   object
 2   conversation  4882 non-null   object
dtypes: int64(1), object(2)
memory usage: 114.6+ KB


,idx,class,conversation
0,2357,협박 대화,이것 좀 먹어봐\n 이게 머야? 이거 먹으면 나 안괴롭히는거야?\n당연하지 이제 우...
1,4831,일반 대화,"다음 달 여행 계획 세웠어?\n응, 제주도로 3박 4일 일정 잡았어!\n좋겠다! 비..."
2,485,갈취 대화,애 아가야\n네? 어머니\n아범 몰래 나 돈 천만 주라\n어머니 제가 그렇게큰돈이 ...
3,1395,협박 대화,난 널 사랑해\n난 이제 너가 너무 무서워\n무슨소리야? 이쁜 입으로 이쁜말만해 입...
4,140,기타 괴롭힘 대화,아저씨 자리 좀 비켜주시겠어요?\n응.? 뭐때문에 그러는건지 알려줄수있니\n아저씨 ...


## 단어 분포 확인

In [7]:
from collections import Counter
from konlpy.tag import Okt

okt = Okt()

# 전처리 함수
def preprocess(text):
    return text.replace('\n', ' ').replace('\t', ' ').strip()

# 품사 태깅
def pos_tagging(text):
    # 관형사, 감탄사, 조사, 어미 등은 불용어 대상으로 토큰화 과정에 제외
    # 부사는 우리의 task에 중요한 정보라 판단하여 포함("진짜 짜증나", "너 정말 못됐다")
    return [word for word, pos in okt.pos(text) if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']]

# 전체 품사별 빈도수 계산
all_tokens = []
for conv in df['conversation']:
    tokens = pos_tagging(preprocess(conv))
    all_tokens.extend(tokens)

all_freq = Counter(all_tokens).most_common(30)
print("전체 상위 단어")
print(all_freq)

# 클래스별 빈도 계산
class_freqs = {}
for label in df['class'].unique():
    class_tokens = []
    sub_df = df[df['class'] == label]
    for conv in sub_df['conversation']:
        tokens = pos_tagging(preprocess(conv))
        class_tokens.extend(tokens)
    class_freqs[label] = Counter(class_tokens).most_common(20)

for cls, freq in class_freqs.items():
    print(f"\n [{cls}] 상위 단어:")
    print(freq)

전체 상위 단어
[('너', 4789), ('내', 4019), ('나', 2878), ('네', 2782), ('말', 2720), ('왜', 2582), ('뭐', 2443), ('거', 2308), ('좀', 2139), ('돈', 1962), ('해', 1807), ('다', 1796), ('있어', 1626), ('것', 1594), ('이', 1581), ('진짜', 1524), ('저', 1488), ('니', 1455), ('죄송합니다', 1447), ('지금', 1446), ('아니', 1398), ('그럼', 1330), ('제', 1297), ('그래', 1269), ('오늘', 1208), ('안', 1151), ('우리', 1148), ('할', 1140), ('생각', 1107), ('어떻게', 1056)]

 [협박 대화] 상위 단어:
[('너', 1519), ('내', 1311), ('말', 722), ('나', 688), ('왜', 621), ('니', 580), ('다', 496), ('제발', 463), ('해', 443), ('이', 422), ('네', 419), ('지금', 399), ('거', 392), ('진짜', 344), ('뭐', 344), ('우리', 328), ('사람', 301), ('그래', 290), ('알', 281), ('안', 279)]

 [일반 대화] 상위 단어:
[('있어', 1131), ('요즘', 829), ('뭐', 648), ('것', 553), ('생각', 470), ('정말', 465), ('같아', 438), ('어떻게', 421), ('오늘', 407), ('수', 395), ('좋은', 389), ('이번', 377), ('응', 367), ('주말', 363), ('해', 359), ('어떤', 324), ('영화', 315), ('나도', 313), ('할', 308), ('계획', 294)]

 [갈취 대화] 상위 단어:
[('돈', 1482), ('너', 970), ('

*전체 상위 단어*
- 너, 내, 나, 네, 우리와 같은 인칭 대명사가 많음
- 이 단어들은 대부분의 클래스에서 공통적으로 많이 등장하며 명확한 구분 정보 없이 자주 쓰이는 표현임
- 몇몇 단어 들은 **불용어 처리**를 고려할 수 있음

*협박 단어*
- 제발, 해, 지금, 진짜, 사람
- “제발 그만해”, “사람 죽이기 전에” 등 감정 호소나 협박 표현이 포함될 수 있음
- 해당 단어들에 높은 가중치를 고려할 수 있음

*일반 대화*
- 요즘, 계획, 영화, 응, 주말, 좋은
- 일상적인 표현이 많이 들어가있음
- 훈련데이터에 없던 합성 데이터인 만큼 **과적합** 우려가 있는 단어

*갈취 대화*
- 돈, 좀, 진짜, 그럼
- "돈 좀 빌려줘" 등 갈취 상황에 쓰이는 표현이 될 수 있음
- 해당 단어들에 높은 **가중치**를 고려할 수 있음

*기타 괴롭힘 대화*
- 다른 클래스에서 사용하는 여러 표현들이 모두 포함돼있음
- 다른 클래스와 구분이 어려울 가능성이 높음
- 주요 특징을 기준으로 **데이터 증강** 등을 통해 충분한 데이터 확보 필요

*직장 내 괴롭힘 대화*
- 죄송합니다, 대리, 회사, 일, 제
- 직급명이나 사과 표현 직장에서 자주 사용하는 표현임
- 해당 단어들에 높은 가중치를 고려할 수 있음
- 다른 클래스에 비해 구분을 잘 할 가능성이 높음


*정리*
- 일반 대화
  - 일상적인 단어가 충분해 다른 클래스와 구분을 잘 하겠지만 합성 데이터이므로 과적합 가능성이 높음

- 갈취 대화
  - "돈"이라는 키워드를 주의깊게 봐야하며 다른 클래스와 구분을 잘 할 가능성이 높음

- 직장 내 괴롭힘 대화
  - 직장에서 사용하는 표현이 자주 있어 다른 클래스와 구분을 잘 할 가능성이 높음

- 협박 대화와 기타 괴롭힘 대화
  - 다른 클래스에서 두루쓰는 표현들이 많아 구분이 어려울 수 있음
  - 데이터 증강이나 주요 표현을 잘 추출하는 방법이 필요